In [1]:
#### Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Handle table-like data and matrices
import numpy as np
import pandas as pd

# Modelling Algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb

# Modelling Helpers
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer , Normalizer , scale, StandardScaler
from sklearn.cross_validation import train_test_split , StratifiedKFold, cross_val_score
from sklearn.feature_selection import RFECV, SelectKBest, SelectFromModel
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report,precision_recall_curve,roc_curve,make_scorer
from sklearn.model_selection import KFold


# Deeplearning tools
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.utils import to_categorical

# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# Configure visualisations
%matplotlib inline
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 8 , 6

# Others
import datetime
import os
import time

/home/chenhao/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/chenhao/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using TensorFlow backend.


In [2]:
pwd

'/home/chenhao/proj/data_proj/stock-data-process'

In [3]:
train_original = pd.read_csv('../stock-data/data-1533016425769.csv')
train_original.head(10)

,id,symbol,frequency,begin_of_bar,end_of_bar,open_price,close_price,low_price,hi_price,volume,amount,is_peak,is_valley,y,gmt_created
0,212566,SHSE.600497,900s,2016-01-04 09:30:00,2016-01-04 09:45:00,5.403103,5.403103,5.348826,5.442578,2556000,27901144,0,0,0,2018-07-30 17:02:36.261914
1,212567,SHSE.600503,900s,2016-01-04 09:30:00,2016-01-04 09:45:00,13.777168,13.429009,13.369324,13.777168,11300800,154109520,0,0,0,2018-07-30 17:02:36.376703
2,212568,SHSE.601238,900s,2016-01-04 09:30:00,2016-01-04 09:45:00,15.434888,15.489597,15.434888,15.619532,215600,4893348,0,0,0,2018-07-30 17:02:36.460579
3,212569,SHSE.600570,900s,2016-01-04 09:30:00,2016-01-04 09:45:00,59.887718,57.990288,57.328163,60.006310,3464800,205674736,0,0,0,2018-07-30 17:02:36.622501
4,212570,SHSE.600737,900s,2016-01-04 09:30:00,2016-01-04 09:45:00,13.956138,13.888854,13.821573,13.956138,1953000,28227520,0,0,0,2018-07-30 17:02:36.766972
5,212571,SHSE.600466,900s,2016-01-04 09:30:00,2016-01-04 09:45:00,9.768019,9.589167,9.547894,9.768019,2468800,34581656,0,0,0,2018-07-30 17:02:36.869707
6,212572,SHSE.600482,900s,2016-01-04 09:30:00,2016-01-04 09:45:00,46.995907,46.112709,46.024387,47.437504,1765600,84095440,0,0,0,2018-07-30 17:02:36.992882
7,212573,SHSE.600276,900s,2016-01-04 09:30:00,2016-01-04 09:45:00,26.076290,25.720463,25.587692,26.076290,935600,45382620,0,0,0,2018-07-30 17:02:37.11213
8,212574,SHSE.600827,900s,2016-01-04 09:30:00,2016-01-04 09:45:00,17.433502,17.268576,17.181263,17.433502,1189200,21200996,0,0,0,2018-07-30 17:02:37.265169
9,212575,SHSE.600016,900s,2016-01-04 09:30:00,2016-01-04 09:45:00,7.236410,7.198244,7.182976,7.282210,16974100,160790736,0,0,0,2018-07-30 17:02:37.355675


In [5]:
train_original.describe()

,id,open_price,close_price,low_price,hi_price,volume,amount,is_peak,is_valley,y
count,1.652926e+06,1.652926e+06,1.652926e+06,1.652926e+06,1.652926e+06,1.652926e+06,1.652926e+06,1.652926e+06,1.652926e+06,1.652926e+06
mean,1.039028e+06,1.802252e+01,1.802389e+01,1.796430e+01,1.808221e+01,2.638859e+06,3.066176e+07,1.493533e-02,2.673441e-02,-2.010374e-03
std,4.771588e+05,3.881074e+01,3.881405e+01,3.870823e+01,3.891727e+01,5.542340e+06,5.916063e+07,1.212942e-01,1.613062e-01,1.040415e-01
min,2.125660e+05,1.455166e+00,1.455166e+00,1.455166e+00,1.465132e+00,1.000000e+00,9.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00
25%,6.257972e+05,6.982121e+00,6.983147e+00,6.960654e+00,7.004458e+00,5.161900e+05,7.291982e+06,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.039028e+06,1.167235e+01,1.167242e+01,1.163231e+01,1.171537e+01,1.171300e+06,1.505177e+07,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.452260e+06,1.891984e+01,1.892000e+01,1.885312e+01,1.898446e+01,2.716151e+06,3.217268e+07,0.000000e+00,0.000000e+00,0.000000e+00
max,1.865491e+06,7.921100e+02,7.919424e+02,7.902761e+02,7.922580e+02,1.109784e+09,1.001395e+10,1.000000e+00,1.000000e+00,1.000000e+00
